### In this file, we will reconstruct our data into graphs

Instructions: You should probably have another separate notebook that creates the graph version of the dataset. Again, you should save the data, and for this make use to use the save_graph and load_graph functions of DGL.

Important Libraries:
ase
Structure/Geometry of a molecule

In [5]:
!pip install ase

Defaulting to user installation because normal site-packages is not writeable
     |████████████████████████████████| 2.2 MB 7.3 MB/s            
     |████████████████████████████████| 11.2 MB 46.3 MB/s            
     |████████████████████████████████| 3.1 MB 55.9 MB/s            
     |████████████████████████████████| 1.6 MB 70.1 MB/s            
     |████████████████████████████████| 873 kB 65.4 MB/s            
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.
  Consider adding this directory to PATH or, if you prefer to suppress this warning, use --no-warn-script-location.


In [41]:
import pandas as pd
import torch
from rdkit import Chem
from rdkit.Chem import AllChem
from ase.io import read, write
from dgllife.utils import smiles_to_bigraph, featurizers
from dgl.data.utils import save_graphs, load_graphs

import os

### Explanation of features:

In [57]:
# atom_type_one_hot -- provides the type of an atom via a list of one-hot encodings; this will allow our model to know what atom type is associated with the other feature values of a given atom
# atomic_number -- provides the atomic number of an atom; similar to our atom type one-hot encodings, this will help our model understand the type of the atom associated with the additional given feature values for that atom
# atom_is_aromatic -- indicates whether an atom is aromatic; this lets our model know whether a given atom is part of a ring bonding structure, i.e, a chain of unhybridized p-orbitals, which proves useful when analyzing smiles strings
# atom_hybridization_one_hot -- provides the hybridization type of an atom via a list of one-hot encodings; this feature is useful in conjunction with the atom_is_aromatic feature as it gives the model more insight into types of orbitals and bonding structures
# atom_degree -- gives the degree of an atom; this lets the model know the number of atoms (not including hydrogens) bonded to a given atom, allowing the model to better understand a given molecule's bonding
# atom_total_degree -- gives the degree of an atom, this time including hydrogens; using this feature in conjunction with atom_degree informs the model about how many atoms are bonded to a particular atom, and how many of those atoms are/are not hydrogens
# atom_explicit_valence -- provides the explicit valence of an atom; this informs the model about how many hydrogens are EXPLICITLY bonded to a given atom
# atom_implicit_valence -- provides the implicit valence of an atom; this informs the model about how many hydrogens are IMPLICITLY bonded to a given atom
# atom_total_num_H -- provides the number of Hs bonded to an atom; used in conjunction with atom_total_degree, atom_explicit_valence, and atom_implicit_valence, this feature further enhances our model's understanding of the bonding of each atom in a given molecule
# atom_formal_charge -- provides the formal charge of an atom; this gives our model additional insight into an atom's number of valence electrons, number of nonbonding valence electrons, and the number of electrons shared in the atom's bonds
# atom_num_radical_electrons -- provides the number of radical electrons of an atom; this lets the model know how many of an atom's valence electrons are unpaired, which is useful because it gives the model insight into how chemically reactive the atom/overall molecule is
# atom_is_in_ring -- indicates whether an atom is part of a ring; this informs the model about whether the atom is part of a ring bonding structure, which is useful given that an atom's hybridization is another feature we've included for the model to consider
# atom_potential_energy -- indicates the minimum potential energy of a given atom; this feature is useful as it enables our model to consider the sum of the minimum potential energies of all atoms in a given molecule
# atom_mass -- gives the mass of an atom (scaled to prevent values from being unnecessarily large); we include this feature in case the model might find correlations between the values of an atom's mass and its other features
# atom position -- indicates the position of an atom in 3D cartesian space; this feature is useful as it gives our model insight into how the spacing between atoms and the lengths of bonds between a molecule's atoms are some of the determinants the molecule's potential energy

In [62]:

# Features added from this paper (https://arxiv.org/pdf/1704.01212.pdf) Table 1
'''
According to table 1 of that paper, these are the features they use. We have already discovered the importance of
both acceptor/donator of electrons as well as partial charges, thus we will for sure want to use those similar features

Unfortunately, we could not get the acceptor/donor binary features in time, though we looked at the WeaveAtomFeaturizer to accomplish this

Atom type        H, C, N, O, F (one-hot)
Atomic number    Number of protons (integer)
Acceptor         Accepts electrons (binary)
Donor            Donates electrons (binary)
Aromatic         In an aromatic system (binary)
Hybridization    sp, sp2, sp3 (one-hot or null)
Number of Hydrogens (integer)
'''

# To get our ROMol and get numhacceptors/donors working, would use PandasTools.AddMoleculeColumnToFrame(esol_data, smilesCol='smiles')

def one_hot_to_value(one_hot_li):
        for i in range(len(one_hot_li)):
            if one_hot_li[i]:
                return i
        return -1
    
def get_energy(atom, atom_to_energy):
    if atom not in atom_to_energy.keys():
        try:
            at = Chem.MolFromSmiles(atom)
            AllChem.EmbedMolecule(at)
            Chem.rdmolfiles.MolToXYZFile(atom_new, 'atom.xyz')
            at = read("atom.xyz")
            at.calc = MOPAC(label='TMP', task='UHF BONDS GRADS')
            atom_potential_energy = at.get_potential_energy()
            
            atom_to_energy[atom] = atom_potential_energy
        except:
            atom_to_energy[atom] = 0
    
    return atom_to_energy[atom]
    

def featurize_atoms(mol):

    atom_to_energy = {}
    atom_to_energy['H'] = -15.47146
    feature_map = lambda atom, xyz, energy: [
                                one_hot_to_value(featurizers.atom_type_one_hot(atom)), # One-hot to index-value of atom type
                                atom.GetAtomicNum(),
#                                 Chem.Descriptors.NumHAcceptors(atom),    # H-Bond acceptors
#                                 Chem.Descriptors.NumHDonors(atom),    # H-Bond donors
                                atom.GetIsAromatic(),
                                one_hot_to_value(featurizers.atom_hybridization_one_hot(atom)), # Hybridization mentioned above
                                atom.GetDegree(),
                                atom.GetTotalDegree(),
                                atom.GetExplicitValence(),
                                atom.GetImplicitValence(),
                                atom.GetTotalNumHs(),
                                atom.GetFormalCharge(),
                                atom.GetNumRadicalElectrons(),
                                atom.IsInRing(), 
                                atom.GetMass() * 0.01,
#                                 energy,
                                xyz[0],
                                xyz[1],
                                xyz[2]
                                ]
    
    feats = []
    AllChem.EmbedMolecule(mol)
    Chem.rdmolfiles.MolToXYZFile(mol, "3Dembedded.xyz")
    empty = False
    if os.stat("3Dembedded.xyz").st_size == 0:
        print('File is empty with mol', mol)
        empty = True
    else:
        mol_ase = read("3Dembedded.xyz")
        pos = mol_ase.get_positions()
    atom_count = 0
    for atom in mol.GetAtoms():
        if empty:
            feats.append(feature_map(atom, [0,0,0], 0))
        else:
            energy = get_energy(atom.GetSymbol(), atom_to_energy)
            feats.append(feature_map(atom, pos[atom_count], energy))
        atom_count += 1
    return {'atom_feats': torch.tensor(feats).reshape(-1, len(feats[0])).float()}

In [61]:
def featurize_bonds(mol):
    feats = []
    bond_types = [Chem.rdchem.BondType.SINGLE, Chem.rdchem.BondType.DOUBLE,
                  Chem.rdchem.BondType.TRIPLE, Chem.rdchem.BondType.AROMATIC]
    for bond in mol.GetBonds():
        btype = bond_types.index(bond.GetBondType())
        is_conjugated = bond.GetIsConjugated()
        is_in_ring = bond.IsInRing()
        stereo_config = bond.GetStereo()
        direction = bond.GetBondDir()
        feats.extend([btype, btype])
        feats.extend([is_conjugated, is_conjugated])
        feats.extend([is_in_ring, is_in_ring])
        feats.extend([stereo_config, stereo_config])
        feats.extend([direction, direction])
    return {'bond_feats': torch.tensor(feats).reshape(-1, 5)}

### Generate Graph objects from our Dataset(s)

In [44]:
from tqdm.notebook import tqdm

In [54]:
def get_graphs(dataset_name):
    df = pd.read_csv(dataset_name)
    graphs =[]
    for smile in tqdm(df["SMILES"]):
        graphs.append(smiles_to_bigraph(smile,
                                       node_featurizer = featurize_atoms,
                                       edge_featurizer = featurize_bonds,
                                       explicit_hydrogens = True)
                     )
    return graphs

In [55]:
graphs = get_graphs("Data/pe_data_combined.csv")

  0%|          | 0/13478 [00:00<?, ?it/s]

File is empty with mol <rdkit.Chem.rdchem.Mol object at 0x7ff92fbeee20>


In [51]:
# Visualize the nodes, the first column of atom features
graphs[0].nodes(), graphs[0].ndata['atom_feats'][:,0]

(tensor([ 0,  1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
         18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35,
         36, 37, 38, 39, 40, 41], dtype=torch.int32),
 tensor([1., 1., 6., 1., 1., 6., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1., 1.,
         1., 1., 1., 1., 7., 6., 7., 6., 6., 6., 6., 6., 8., 6., 6., 6., 6., 6.,
         6., 7., 6., 6., 6., 7.]))

In [52]:
graphs[-1]

Graph(num_nodes=24, num_edges=46,
      ndata_schemes={'atom_feats': Scheme(shape=(11,), dtype=torch.float32)}
      edata_schemes={'bond_feats': Scheme(shape=(5,), dtype=torch.int64)})

In [54]:
graphs[0].edges()[0], graphs[0].edata['bond_feats'][:4]

(tensor([24, 31, 31, 30, 31, 32, 32, 29, 29, 22, 22, 23, 23, 37, 37, 27, 27, 28,
         28, 35, 35, 36, 36, 38, 38, 39, 39, 25, 25, 26, 36, 33, 33, 34, 34, 40,
         40, 41, 41,  2,  2,  5, 37, 32, 26, 35,  5, 33, 33, 23, 24,  1, 24,  4,
         29, 14, 27, 13, 27, 15, 28, 12, 28, 10, 38, 11, 39,  6, 25,  3, 26,  0,
         34, 19, 34,  7, 40,  9, 40, 18, 41, 20,  2,  8,  2, 17,  5, 21,  5, 16],
        dtype=torch.int32),
 tensor([[0, 0, 1, 1, 0],
         [0, 0, 0, 0, 0],
         [1, 1, 1, 1, 0],
         [0, 0, 0, 0, 0]]))

In [56]:
save_graphs("./DataGraphs/combined_graph_low_features.bin", graphs)

In [57]:
gs = load_graphs("./DataGraphs/pe_data_combined_graph.bin")[0][0]
gs.number_of_nodes

<bound method DGLHeteroGraph.number_of_nodes of Graph(num_nodes=42, num_edges=90,
      ndata_schemes={'atom_feats': Scheme(shape=(11,), dtype=torch.float32)}
      edata_schemes={'bond_feats': Scheme(shape=(5,), dtype=torch.int64)})>